# Baseline Model

L’objectif de ce notebook est de mettre en place un **modèle de référence (baseline)** pour le projet de détection de fraude. Ce modèle servira de point de comparaison pour toutes les améliorations futures et permettra d’évaluer rapidement la performance des différentes approches.

Pour cette étape initiale, nous nous concentrons sur trois algorithmes de **gradient boosting**, réputés pour leur efficacité sur des problèmes de classification avec des données structurées :

1. **XGBoost** – un modèle performant et largement utilisé pour les problèmes de classification et de régression.  
2. **LightGBM** – optimisé pour les grands ensembles de données, offrant rapidité et efficacité mémoire.  
3. **CatBoost** – particulièrement efficace pour gérer les variables catégorielles sans avoir besoin d’un encodage manuel complexe.

Ce notebook inclura :  
- La préparation des données (encodage des variables catégorielles, sélection des features).  
- L’entraînement de chaque modèle sur les données d’entraînement.  
- L’évaluation des modèles avec des métriques adaptées à la détection de fraude (précision, rappel, F1-score, AUC-ROC).  
- Une comparaison des performances pour identifier le modèle de référence le plus performant sur cet ensemble de données.


In [5]:
import pandas as pd
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier


In [4]:
df_train = pd.read_csv('../Data/train_reduced.csv')

TypeError: unsupported operand type(s) for +: 'numpy.dtypes.Int64DType' and 'numpy.dtypes.Int64DType'